<a href="https://colab.research.google.com/github/christianloring/Thinkful-assignments/blob/main/evaluating_performance_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
import statsmodels.formula.api as smf
from sqlalchemy import create_engine
import statsmodels.api as sm

# Display preferences
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings(action="ignore")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# No need for an open connection, because you're only doing a single query
engine.dispose()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Target is diff between temp and apparenttemp
# Explained by humidity and windspeed
Y = weather_df["temperature"] - weather_df["apparenttemperature"]
X = weather_df[["humidity", "windspeed"]]
# Fit method estimates the coefficients using OLS
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
# Inspect the results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Fri, 26 Nov 2021   Prob (F-statistic):               0.00
Time:                        05:09:54   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4381      0.021   -115.948      0.0

R sq = adj R sq = 0.288.
This seems unsatisfactory, values between 0.7-0.9 would be more ideal and this is far from that.

In [ ]:
# Target is diff between temp and apparenttemp
# Explained by humidity, windspeed, and their interaction
weather_df["humid_windspeed"] = weather_df["humidity"] * weather_df["windspeed"]
Y = weather_df["temperature"] - weather_df["apparenttemperature"]
X = weather_df[["humidity", "windspeed", "humid_windspeed"]]
# Fit method estimates the coefficients using OLS
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
# Inspect the results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Fri, 26 Nov 2021   Prob (F-statistic):               0.00
Time:                        05:11:55   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0839      0.033     

r^2 = adj r^2 = 0.341, which is better but still not good enough. The model is improved.

In [ ]:
# Target is diff between temp and apparenttemp
# Explained by humidity, visibility and windspeed
Y = weather_df["temperature"] - weather_df["apparenttemperature"]
X = weather_df[["humidity", "windspeed", "visibility"]]
# Fit method estimates the coefficients using OLS
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
# Inspect the results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Fri, 26 Nov 2021   Prob (F-statistic):               0.00
Time:                        05:13:07   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.5756      0.028    -56.605      0.0

Better than first, worse than second. The interaction is better as it explains ~4% more of the unexplained variance compared to the one with visibility. The second model has the best AIC and BIC scores.